In [1]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D
import numpy as np

import pandas as pd

import keras

df = pd.read_csv('/content/drive/MyDrive/stocks/ADANIPORTS.csv')

print(df.columns)

Index(['Date', 'Symbol', 'Series', 'Prev Close', 'Open', 'High', 'Low', 'Last',
       'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume',
       '%Deliverble'],
      dtype='object')


In [2]:
features = df[['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Turnover', '%Deliverble', 'Deliverable Volume']].values
prices = df['Close'].values.reshape(-1,1)

# Scale the features
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaled_features = scaler_x.fit_transform(features)
scaled_prices = scaler_y.fit_transform(prices)

In [3]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
principal_components = pca.fit_transform(scaled_features)

print(principal_components.shape)
print(features.shape)

(3322, 4)
(3322, 9)


In [4]:
# Update create_sequences function to handle multiple features
def create_sequences(features, prices, sequence_length):
    sequences = []
    labels = []
    for i in range(len(features) - (sequence_length + 30)):
        sequences.append(features[i:i + sequence_length])
        labels.append(prices[i+sequence_length+30])  # predict the 'Close' price
    return np.array(sequences), np.array(labels)

sequence_length = 60
X, y = create_sequences(principal_components, scaled_prices, sequence_length)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.callbacks import EarlyStopping, LearningRateScheduler

# Building the LSTM model
model = Sequential([
    Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.1),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
def scheduler(epoch, lr):
    if (epoch) % 50 == 0:
        return lr * 0.8  # Reduce by 20% after each 50 epochs
    else:
        return lr

lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = model.fit(X_train, y_train, epochs=450, batch_size=32, validation_split=0.1, callbacks=[early_stopping, lr_scheduler])

Epoch 1/450
73/73 [==============================] - 14s 113ms/step - loss: 0.0097 - val_loss: 0.0053 - lr: 8.0000e-04
Epoch 2/450
73/73 [==============================] - 7s 99ms/step - loss: 0.0063 - val_loss: 0.0045 - lr: 8.0000e-04
Epoch 3/450
73/73 [==============================] - 7s 97ms/step - loss: 0.0058 - val_loss: 0.0039 - lr: 8.0000e-04
Epoch 4/450
73/73 [==============================] - 7s 97ms/step - loss: 0.0055 - val_loss: 0.0032 - lr: 8.0000e-04
Epoch 5/450
73/73 [==============================] - 7s 98ms/step - loss: 0.0051 - val_loss: 0.0031 - lr: 8.0000e-04
Epoch 6/450
73/73 [==============================] - 7s 96ms/step - loss: 0.0039 - val_loss: 0.0022 - lr: 8.0000e-04
Epoch 7/450
73/73 [==============================] - 7s 97ms/step - loss: 0.0042 - val_loss: 0.0025 - lr: 8.0000e-04
Epoch 8/450
73/73 [==============================] - 7s 96ms/step - loss: 0.0037 - val_loss: 0.0021 - lr: 8.0000e-04
Epoch 9/450
73/73 [==============================] - 7s 98ms/s

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

p_test = model.predict(X_test)
p_test = scaler_y.inverse_transform(p_test)
actual_prices = scaler_y.inverse_transform(y_test.reshape(-1,1))

mae = mean_absolute_error(actual_prices, p_test)
mse = mean_squared_error(actual_prices, p_test)
rmse = root_mean_squared_error(actual_prices, p_test)
r2 = r2_score(actual_prices, p_test)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')



21/21 [==============================] - 2s 36ms/step
Mean Absolute Error (MAE): 11.288748475536128
Mean Squared Error (MSE): 758.1176102064554
Root Mean Squared Error (RMSE): 27.533935610559844
R-squared (R²): 0.9763827229901711


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
principal_components = pca.fit_transform(scaled_features)


# Update create_sequences function to handle multiple features
def create_sequences(features, prices, sequence_length):
    sequences = []
    labels = []
    for i in range(len(features) - (sequence_length + 30)):
        sequences.append(features[i:i + sequence_length])
        labels.append(prices[i+sequence_length+30])  # predict the 'Close' price
    return np.array(sequences), np.array(labels)

sequence_length = 60
X, y = create_sequences(principal_components, scaled_prices, sequence_length)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from keras.callbacks import EarlyStopping, LearningRateScheduler

# Building the LSTM model
model = Sequential([
    Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.1),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
def scheduler(epoch, lr):
    if (epoch) % 40 == 0:
        return lr * 0.8  # Reduce by 20% after each 50 epochs
    else:
        return lr

lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = model.fit(X_train, y_train, epochs=450, batch_size=32, validation_split=0.1, callbacks=[early_stopping, lr_scheduler])

Epoch 1/450
73/73 [==============================] - 14s 118ms/step - loss: 0.0094 - val_loss: 0.0055 - lr: 8.0000e-04
Epoch 2/450
73/73 [==============================] - 7s 96ms/step - loss: 0.0062 - val_loss: 0.0042 - lr: 8.0000e-04
Epoch 3/450
73/73 [==============================] - 7s 99ms/step - loss: 0.0066 - val_loss: 0.0043 - lr: 8.0000e-04
Epoch 4/450
73/73 [==============================] - 7s 99ms/step - loss: 0.0055 - val_loss: 0.0035 - lr: 8.0000e-04
Epoch 5/450
73/73 [==============================] - 7s 98ms/step - loss: 0.0050 - val_loss: 0.0029 - lr: 8.0000e-04
Epoch 6/450
73/73 [==============================] - 7s 99ms/step - loss: 0.0044 - val_loss: 0.0031 - lr: 8.0000e-04
Epoch 7/450
73/73 [==============================] - 7s 97ms/step - loss: 0.0040 - val_loss: 0.0022 - lr: 8.0000e-04
Epoch 8/450
73/73 [==============================] - 7s 100ms/step - loss: 0.0034 - val_loss: 0.0021 - lr: 8.0000e-04
Epoch 9/450
73/73 [==============================] - 7s 98ms/

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

p_test = model.predict(X_test)
p_test = scaler_y.inverse_transform(p_test)
actual_prices = scaler_y.inverse_transform(y_test.reshape(-1,1))

mae = mean_absolute_error(actual_prices, p_test)
mse = mean_squared_error(actual_prices, p_test)
rmse = root_mean_squared_error(actual_prices, p_test)
r2 = r2_score(actual_prices, p_test)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')



21/21 [==============================] - 2s 39ms/step
Mean Absolute Error (MAE): 12.61146488808775
Mean Squared Error (MSE): 687.7541684584768
Root Mean Squared Error (RMSE): 26.225067558701863
R-squared (R²): 0.9785747217945182


In [9]:
from keras.callbacks import EarlyStopping, LearningRateScheduler

# Building the LSTM model
model = Sequential([
    Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.1),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
def scheduler(epoch, lr):
    if (epoch) % 40 == 0:
        return lr * 0.8  # Reduce by 20% after each 50 epochs
    else:
        return lr

lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = model.fit(X_train, y_train, epochs=450, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/450
73/73 [==============================] - 16s 140ms/step - loss: 0.0083 - val_loss: 0.0049
Epoch 2/450
73/73 [==============================] - 9s 124ms/step - loss: 0.0068 - val_loss: 0.0046
Epoch 3/450
73/73 [==============================] - 9s 121ms/step - loss: 0.0057 - val_loss: 0.0035
Epoch 4/450
73/73 [==============================] - 9s 120ms/step - loss: 0.0049 - val_loss: 0.0033
Epoch 5/450
73/73 [==============================] - 9s 120ms/step - loss: 0.0047 - val_loss: 0.0027
Epoch 6/450
73/73 [==============================] - 9s 121ms/step - loss: 0.0045 - val_loss: 0.0030
Epoch 7/450
73/73 [==============================] - 9s 120ms/step - loss: 0.0039 - val_loss: 0.0022
Epoch 8/450
73/73 [==============================] - 9s 121ms/step - loss: 0.0034 - val_loss: 0.0022
Epoch 9/450
73/73 [==============================] - 9s 122ms/step - loss: 0.0028 - val_loss: 0.0014
Epoch 10/450
73/73 [==============================] - 9s 120ms/step - loss: 0.0027 - val_l

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

p_test = model.predict(X_test)
p_test = scaler_y.inverse_transform(p_test)
actual_prices = scaler_y.inverse_transform(y_test.reshape(-1,1))

mae = mean_absolute_error(actual_prices, p_test)
mse = mean_squared_error(actual_prices, p_test)
rmse = root_mean_squared_error(actual_prices, p_test)
r2 = r2_score(actual_prices, p_test)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')



21/21 [==============================] - 2s 48ms/step
Mean Absolute Error (MAE): 11.552073537363704
Mean Squared Error (MSE): 528.4121393406796
Root Mean Squared Error (RMSE): 22.9872168680917
R-squared (R²): 0.9835386281730528
